# 2.1 Using Tools with Semantic Kernel

By adding functions (tools) to your kernel, the model gets information about those functions, and it will tell you that it wants to execute a function. The kernel can automatically handle function execution and return results back to the LLM. This is similar to LangChain's tool binding but uses Semantic Kernel's plugin system.

In [ ]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from typing import Annotated

Create the kernel and configure Azure OpenAI service

In [ ]:
# Create a kernel instance
kernel = Kernel()

# Add Azure OpenAI chat completion service
service_id = "azure_openai"
kernel.add_service(
    AzureChatCompletion(
        service_id=service_id,
        deployment_name="gpt-4.1-mini",
    )
)

Define the weather tool as a Semantic Kernel function

In [ ]:
class WeatherPlugin:
    """
    A plugin that provides weather information for specific cities.
    """
    
    @kernel_function(
        description="Get the current weather for a specified location",
        name="get_weather"
    )
    def get_weather(
        self,
        location: Annotated[str, "The name of the city to get the weather for. Must be one of 'Chicago', 'New York', or 'Los Angeles'"]
    ) -> Annotated[str, "A string describing the current weather in the specified location"]:
        """
        Get the current weather for a specified location.
        """
        weather_data = {
            "New York": "Sunny, 25°C",
            "Los Angeles": "Cloudy, 22°C",
            "Chicago": "Rainy, 18°C"
        }
        return weather_data.get(location, "Weather data not available for this location.")

Add the plugin to the kernel

In [ ]:
# Add the weather plugin to the kernel
kernel.add_plugin(WeatherPlugin(), plugin_name="weather")

Set up chat history with system and user messages

In [ ]:
# Create chat history
chat_history = ChatHistory()

# Add system and user messages
chat_history.add_system_message(
    "You are a helpful assistant that can provide weather information for specific cities."
)
chat_history.add_user_message("What is the weather like in NYC?")

Configure function calling and get response

In [ ]:
# Configure execution settings to enable automatic function calling
execution_settings = kernel.get_prompt_execution_settings_from_service_id(service_id)
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Get the chat completion service
chat_completion = kernel.get_service(service_id)

# Generate response with automatic function calling
response = await chat_completion.get_chat_message_contents(
    chat_history=chat_history,
    settings=execution_settings,
    kernel=kernel  # Pass kernel to enable function calling
)

# Add the response to chat history
chat_history.add_message(response[0])

Print the conversation

In [ ]:
# Print the conversation
for message in chat_history.messages:
    role = message.role.value.title()
    content = str(message.content)
    print(f"================================ {role} Message ================================")
    print()
    print(content)
    print()

Notice how the function call and execution happens automatically when `FunctionChoiceBehavior.Auto()` is used. The kernel handles:
1. Function discovery and schema generation
2. Function calling by the LLM
3. Function execution
4. Result integration back into the conversation

**NOTE:** Usage of functions means multiple LLM calls may happen automatically - one to determine what function to call and potentially more to process the results.

## Key Differences from LangChain

1. **Plugin System**: Semantic Kernel uses a plugin-based architecture where functions are organized into plugins (classes) rather than individual tool decorators.

2. **Kernel Functions**: Use `@kernel_function` decorator instead of `@tool` to define functions available to the LLM.

3. **Automatic Function Calling**: With `FunctionChoiceBehavior.Auto()`, the kernel automatically handles function calling, execution, and result integration.

4. **Type Annotations**: Semantic Kernel leverages Python's `Annotated` type hints for better function schema generation and documentation.

5. **Simplified Flow**: No need to manually check for tool calls or handle tool messages - the kernel manages the entire flow automatically.

6. **Plugin Organization**: Functions are grouped into logical plugins, making it easier to manage and organize capabilities.

This approach provides better abstraction and automation compared to manually handling tool calls in LangChain.